In [2]:
import torch
import time

In [3]:
def det(m):
    size = len(m)
    det = 1
    for row in range(size):
        swap = row+1
        while m[row][row] == 0:
            if (swap == size):
                return 0
            m[row], m[swap] = m[swap], m[row]
            det*=-1
            swap+=1
        for op in range(row+1, size):
            for col in range(row+1, size):
                m[op][col] = m[op][col] - m[row][col] * m[op][row] / m[row][row]
            m[op][row] = 0
    for i in range(size):
        det*=m[i][i]
    return det

def mult(m1, m2):
    dim = len(m1[0])
    if dim != len(m2):
        raise ValueError("Invalid matrix dimensions")
    rows = len(m1)
    cols = len(m2[0])
    result = [[0 for c in range(cols)] for r in range(rows)]
    for r in range(rows):
        for c in range(cols):
            for d in range(dim):
                result[r][c] += m1[r][d]*m2[d][c]
    return result
    
def rand_matrix(size):
    return [[torch.rand(1).item() for i in range(size)] for j in range(size)]

In [17]:
size = 16384

# torch.set_default_tensor_type(torch.FloatTensor)
# torch.set_float32_matmul_precision('highest')
torch.backends.cuda.matmul.allow_tf32 = True

for i in range(0, 1):
    c1 = torch.randn(size, size, dtype=torch.float32)
    c2 = torch.randn(size, size, dtype=torch.float32)

    start = time.time()
    g1 = c1.to("cuda")
    g2 = c2.to("cuda")
    end = time.time()
    print("to cuda: " + str((end-start)*1000000))

    start = time.time()
    for i in range(0, 100):
      r2 = g1 @ g2
    torch.cuda.synchronize()
    end = time.time()
    print("mult: " + str((end-start)*1000000))

    start = time.time()
    r_cpu = r2.cpu()
    end = time.time()
    print("to cpu: " + str((end-start)*1000000))

to cuda: 114657.40203857422
mult: 107898.23532104492
to cpu: 288570.1656341553
